In [135]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelBinarizer

In [136]:
# import nltk
# nltk.download()

In [137]:
train = pd.read_csv(".\\data\\train.csv", encoding='utf=8')
test = pd.read_csv(".\\data\\test.csv", encoding='utf=8')
print train.shape
print test.shape

(38932, 5)
(29404, 4)


In [138]:
train_len = train.shape[0]
print train_len

38932


In [139]:
train_labels = train["Is_Response"]
train.drop("Is_Response", axis=1, inplace=True)
print train.shape
print train.columns

(38932, 4)
Index([u'User_ID', u'Description', u'Browser_Used', u'Device_Used'], dtype='object')


In [140]:
total_data = train.append(test)
print total_data.shape

(68336, 4)


In [141]:
cols = ['Browser_Used', 'Device_Used']
for c in cols:
    lb = LabelEncoder()
    total_data[c] = lb.fit_transform(total_data[c])
    print lb.classes_

[u'Chrome' u'Edge' u'Firefox' u'Google Chrome' u'IE' u'Internet Explorer'
 u'InternetExplorer' u'Mozilla' u'Mozilla Firefox' u'Opera' u'Safari']
[u'Desktop' u'Mobile' u'Tablet']


In [142]:
# stemmer = PorterStemmer()
# # stop_words = set(stopwords.words('english'))
# for i,r in total_data.iterrows():
#     wt = word_tokenize(r[1])
#     filtered_desc = []
#     for w in wt:
#         filtered_desc.append(stemmer.stem(w))
#     print i,
#     total_data.loc[i,"Description"] = " ".join(filtered_desc)

In [143]:
stemmed_df = pd.read_csv("stemmed.csv", encoding='utf=8')
total_data["Description"] = stemmed_df["Description"]
# total_data.to_csv("stemmed.csv", index=False,encoding ='utf-8')

In [144]:
train_new = total_data.iloc[:train_len,:]
submission = total_data.iloc[train_len:,:]
submission_user_id = submission["User_ID"]
train_new.drop("User_ID", axis=1, inplace=True)
submission.drop("User_ID", axis=1, inplace=True)
print train_new.shape
print submission.shape

D:\sasnjeev\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(38932, 3)
(29404, 3)


D:\sasnjeev\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [145]:
print train_new.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_new,train_labels, test_size=0.33, random_state=2017)

(38932, 3)


In [146]:
print X_train.shape
print X_train.columns
print X_test.shape
print X_test.columns
print y_train.shape
# print y_train.columns
print y_test.shape

(26084, 3)
Index([u'Description', u'Browser_Used', u'Device_Used'], dtype='object')
(12848, 3)
Index([u'Description', u'Browser_Used', u'Device_Used'], dtype='object')
(26084L,)
(12848L,)


In [147]:
cv = CountVectorizer(analyzer='word',stop_words='english', min_df=150, ngram_range=(1,1), max_features=700)
cv.fit(X_train.Description.values.tolist()+X_test.Description.values.tolist()+submission.Description.values.tolist())
tfidf = TfidfVectorizer(analyzer='word',stop_words='english', min_df=150, ngram_range=(1,1), max_features=700)
tfidf.fit(X_train.Description.values.tolist()+X_test.Description.values.tolist()+submission.Description.values.tolist())

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=700, min_df=150,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [148]:
X_train_desc_vec = cv.transform(X_train.Description.values.tolist())
X_test_desc_vec = cv.transform(X_test.Description.values.tolist())
submission_desc_vec = cv.transform(submission.Description.values.tolist())

In [149]:
X_train_desc_tfidf = tfidf.transform(X_train.Description.values.tolist())
X_test_desc_tfidf = tfidf.transform(X_test.Description.values.tolist())
submission_desc_tfidf = tfidf.transform(submission.Description.values.tolist())

In [150]:
lb_labels = LabelBinarizer()
# print y_train[:5]
y_train = lb_labels.fit_transform(y_train)
y_test = lb_labels.transform(y_test)
print lb_labels.classes_

[u'happy' u'not happy']


In [151]:
xtrain = hstack([csr_matrix(X_train[cols].astype(float).values), X_train_desc_vec, X_train_desc_tfidf])
xtest = hstack([csr_matrix(X_test[cols].astype(float).values), X_test_desc_vec, X_test_desc_tfidf])
xsubmission = hstack([csr_matrix(submission[cols].astype(float).values), submission_desc_vec, submission_desc_tfidf])

In [152]:
y_train_ravel = np.array(y_train).ravel()
y_test_ravel = np.array(y_test).ravel()
# print type(y_train)
# print type(y_test)
y_train_ravel

array([0, 1, 1, ..., 0, 1, 0])

In [153]:
# dtrain = xgb.DMatrix(xtrain, label=y_train)
# dtest = xgb.DMatrix(xtest, label=y_test)
# dsubmission = xgb.DMatrix(xsubmission)
dtrain = lgb.Dataset(xtrain, label=y_train_ravel)
dtest = lgb.Dataset(xtest, label=y_test_ravel, reference=dtrain)
dsubmission = lgb.Dataset(xsubmission)

In [160]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'num_leaves': 64,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1,
    'max_bin': 512,
    'subsample_for_bin': 200,
    'subsample': 1, 
    'subsample_freq': 1, 
    'colsample_bytree': 0.8, 
    'reg_alpha': 5, 
    'reg_lambda': 10,
    'min_split_gain': 0.5, 
    'min_child_weight': 1, 
    'min_child_samples': 5, 
    'scale_pos_weight': 1,
    }
gbm = lgb.train(params, dtrain, num_boost_round=200, valid_sets=dtest, early_stopping_rounds=10)

[1]	valid_0's binary_error: 0.22486
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_error: 0.212407
[3]	valid_0's binary_error: 0.199875
[4]	valid_0's binary_error: 0.194505
[5]	valid_0's binary_error: 0.190613
[6]	valid_0's binary_error: 0.188745
[7]	valid_0's binary_error: 0.188745
[8]	valid_0's binary_error: 0.184854
[9]	valid_0's binary_error: 0.185632
[10]	valid_0's binary_error: 0.182674
[11]	valid_0's binary_error: 0.180729
[12]	valid_0's binary_error: 0.180028
[13]	valid_0's binary_error: 0.17707
[14]	valid_0's binary_error: 0.176681
[15]	valid_0's binary_error: 0.176603
[16]	valid_0's binary_error: 0.172634
[17]	valid_0's binary_error: 0.170143
[18]	valid_0's binary_error: 0.16703
[19]	valid_0's binary_error: 0.166252
[20]	valid_0's binary_error: 0.16345
[21]	valid_0's binary_error: 0.163527
[22]	valid_0's binary_error: 0.162983
[23]	valid_0's binary_error: 0.162516
[24]	valid_0's binary_error: 0.161971
[25]	valid_0's binary_error: 0.160181
[

In [161]:
pred = gbm.predict(xtest)
print pred[:5]
print y_test[:5]
pred_ = pred>0.5
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred_)
print score

[ 0.31605933  0.03827838  0.06636349  0.03601756  0.36435689]
[[1]
 [0]
 [0]
 [1]
 [0]]
0.878035491905


In [162]:
submit_pred = gbm.predict(xsubmission, num_iteration=gbm.best_iteration)
print submit_pred.shape

(29404L,)


In [163]:
submit_pred = lb_labels.inverse_transform(submit_pred)
submit_df = pd.concat([submission_user_id, pd.DataFrame(submit_pred)], axis=1)
submit_df.columns = ["User_ID", "Is_Response"]
print submit_df.shape

(29404, 2)


In [158]:
print submit_df.head(5)
submit_df.to_csv("submission.csv", index=False)

   User_ID Is_Response
0  id80132   not happy
1  id80133   not happy
2  id80134   not happy
3  id80135       happy
4  id80136   not happy
